In [1]:
import pandas
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tqdm import tqdm

2022-05-23 15:07:17.300727: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# data path
data_path = '/data/linghui/ml-1m/'

# file : ratings.dat  movies.dat  users.dat

user_file = data_path + 'users.dat'
rating_file = data_path + 'ratings.dat'
movie_file = data_path + 'movies.dat'

In [3]:
# load user data
user_data = pd.read_csv(user_file, sep='::', header=None)
user_data.columns = ['user_id',  'gender', 'age','occupation', 'zipcode']
user_data

/opt/conda/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,user_id,gender,age,occupation,zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [4]:
#load rating data

rating_data = pd.read_csv(rating_file, sep='::', header=None)
rating_data.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
rating_data

/opt/conda/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [5]:
# load movie data

movie_data = pd.read_csv(movie_file, sep='::', header=None, error_bad_lines=False,engine='python', encoding="ISO-8859-1")
movie_data.columns = ['movie_id', 'title', 'genres']
movie_data

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [6]:
# add negative sample 

def neg_sample(u_data, neg_rate=1):
    item_ids = u_data['movie_id'].unique()
    print('start neg sample')
    neg_data = []
    for user_id, hist in tqdm(u_data.groupby('user_id')):
        # 
        rated_movie_list = hist['movie_id'].tolist()
        candidate_set = list(set(item_ids) - set(rated_movie_list))
        neg_list_id = np.random.choice(candidate_set, size=len(rated_movie_list) * neg_rate, replace=True)
        for id in neg_list_id:
            neg_data.append([user_id, id, -1, 0])
    u_data_neg = pd.DataFrame(neg_data)
    u_data_neg.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
    u_data = pd.concat([u_data, u_data_neg])
    print('end neg sample')
    return u_data

rating_data = neg_sample(rating_data, neg_rate=1)
rating_data

start neg sample


100%|██████████| 6040/6040 [00:09<00:00, 668.34it/s]


end neg sample


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,2209,-1,0
1000205,6040,1833,-1,0
1000206,6040,3728,-1,0
1000207,6040,1726,-1,0


In [7]:
#merge three data on movie_id , user_id

merge_data = pd.merge(rating_data, movie_data, on="movie_id", how='left')
merge_data = pd.merge(merge_data, user_data, on="user_id", how='left')
merge_data

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zipcode
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067
...,...,...,...,...,...,...,...,...,...,...
2000413,6040,2209,-1,0,Young and Innocent (1937),Crime|Thriller,M,25,6,11106
2000414,6040,1833,-1,0,Mercury Rising (1998),Action|Drama|Thriller,M,25,6,11106
2000415,6040,3728,-1,0,One False Move (1991),Thriller,M,25,6,11106
2000416,6040,1726,-1,0,"Postman, The (1997)",Drama,M,25,6,11106


In [8]:
# choose feature and prepare for labelencoding

sparse_features = ["movie_id", "user_id",
                   "gender", "age", "occupation", "zipcode"]

merge_data[sparse_features] = merge_data[sparse_features].astype(str)
target = ['rating']

In [9]:
#  rating >=0  - label 1  rating = 0 (negative sample) - label 0
select_score = 0
merge_data['rating'] = [1 if int(x) >= select_score else 0 for x in merge_data['rating']]
merge_data

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zipcode
0,1,1193,1,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,1,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,1,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,1,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,1,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067
...,...,...,...,...,...,...,...,...,...,...
2000413,6040,2209,0,0,Young and Innocent (1937),Crime|Thriller,M,25,6,11106
2000414,6040,1833,0,0,Mercury Rising (1998),Action|Drama|Thriller,M,25,6,11106
2000415,6040,3728,0,0,One False Move (1991),Thriller,M,25,6,11106
2000416,6040,1726,0,0,"Postman, The (1997)",Drama,M,25,6,11106


In [10]:
for feat in sparse_features:
    lbe = LabelEncoder()
    merge_data[feat] = lbe.fit_transform(merge_data[feat])

In [11]:
merge_data

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zipcode
0,0,189,1,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,0,0,2,1588
1,0,3374,1,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,0,0,2,1588
2,0,3615,1,978301968,My Fair Lady (1964),Musical|Romance,0,0,2,1588
3,0,2503,1,978300275,Erin Brockovich (2000),Drama,0,0,2,1588
4,0,1374,1,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,0,0,2,1588
...,...,...,...,...,...,...,...,...,...,...
2000413,5602,1224,0,0,Young and Innocent (1937),Crime|Thriller,1,2,17,466
2000414,5602,810,0,0,Mercury Rising (1998),Action|Drama|Thriller,1,2,17,466
2000415,5602,2847,0,0,One False Move (1991),Thriller,1,2,17,466
2000416,5602,721,0,0,"Postman, The (1997)",Drama,1,2,17,466


In [12]:
merge_data.user_id.value_counts()

3522    4628
757     3700
3642    3486
1047    3190
203     3042
        ... 
5251      40
2676      40
739       40
3800      40
2248      40
Name: user_id, Length: 6040, dtype: int64

In [13]:
# encoding movie genres and pad to fixed length

def split(x):
    key_ans = x.split('|')
    for key in key_ans:
        if key not in key2index:
            # Notice : input value 0 is a special "padding",so we do not use 0 to encode valid feature for sequence input
            key2index[key] = len(key2index) + 1
    return list(map(lambda x: key2index[x], key_ans))

In [14]:
key2index = {}
genres_list = list(map(split, merge_data['genres'].values))
genres_length = np.array(list(map(len, genres_list)))
max_len = max(genres_length)
# Notice : padding=`post`
genres_list = pad_sequences(genres_list, maxlen=max_len, padding='post')

In [16]:
len(genres_list)

2000418

In [17]:
merge_data['genres'] = genres_list

In [19]:
saved_data_path = '/data/linghui/ml-1m/processed_data/'
processed_data = saved_data_path + 'ml-1m-negrate-1.csv'
# merge_data.to_csv(processed_data)

In [42]:
train_data , test_data = train_test_split(merge_data,test_size=0.2)

In [43]:
train_data.to_csv(saved_data_path + 'train_data.csv')
test_data.to_csv(saved_data_path + 'test_data.csv')

In [ ]:
from ast import literal_eval


In [75]:
train_data_path = saved_data_path + 'train_data.csv' 
test_data_path = saved_data_path + 'test_data.csv'

#load data
train_data = pd.read_csv(train_data_path,index_col=0)

/opt/conda/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [76]:
genres_list = train_data['genres'].tolist()
genres = []
for i in range(len(genres_list)):
    genres.append(literal_eval(genres_list[i]))

In [78]:
train_data['genres'] = genres

In [79]:
train_data

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zipcode
1595412,2914,1192,0,0,Boys on the Side (1995),"[6, 1, 0, 0, 0, 0]",1,1,15,1181
325297,1030,2554,1,974771606,"Color of Paradise, The (Rang-e Khoda) (1999)","[1, 0, 0, 0, 0, 0]",0,0,0,3193
998295,5594,2591,1,956714059,Clear and Present Danger (1994),"[7, 8, 12, 0, 0, 0]",1,5,5,2463
948417,5254,1862,1,958473217,Mickey Blue Eyes (1999),"[6, 5, 0, 0, 0, 0]",0,3,13,397
1347078,1151,1722,0,0,Buena Vista Social Club (1999),"[18, 0, 0, 0, 0, 0]",0,4,12,61
...,...,...,...,...,...,...,...,...,...,...
407439,1608,2848,1,974236406,Shaft (1971),"[7, 13, 0, 0, 0, 0]",1,1,18,423
1780453,4065,744,0,0,Half Baked (1998),"[6, 0, 0, 0, 0, 0]",1,2,15,3423
545450,2628,2154,1,967669576,Mansfield Park (1999),"[1, 0, 0, 0, 0, 0]",1,6,5,671
961002,5328,1081,1,958145078,Waterworld (1995),"[7, 8, 0, 0, 0, 0]",1,2,1,2976


In [ ]:
# with open('train_data.csv','w', newline='') as csvfile:
#   writer = csv.writer(csvfile)
#   for row in list(y_test):
#     writer.writerow(row)

In [24]:
import sys
deepctr_path = '/root/linghui/rec-fairness/'
sys.path.append(deepctr_path)
import deepctr
from deepctr.models import DeepFEFM
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, get_feature_names

In [26]:
fixlen_feature_columns = [SparseFeat(feat, merge_data[feat].nunique(), embedding_dim=4)
								  for feat in sparse_features]

use_weighted_sequence = False
if use_weighted_sequence:
    varlen_feature_columns = [VarLenSparseFeat(SparseFeat('genres', vocabulary_size=len(
        key2index) + 1, embedding_dim=4), maxlen=max_len, combiner='mean',
                                                weight_name='genres_weight')]  # Notice : value 0 is for padding for sequence input feature
else:
    varlen_feature_columns = [VarLenSparseFeat(SparseFeat('genres', vocabulary_size=len(
        key2index) + 1, embedding_dim=4), maxlen=max_len, combiner='mean',
                                                weight_name=None)]  # Notice : value 0 is for padding for sequence input feature

linear_feature_columns = fixlen_feature_columns + varlen_feature_columns
dnn_feature_columns = fixlen_feature_columns + varlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)